In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential ,load_model
from tensorflow.keras.layers import LSTM,Dense,Dropout
import os

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
df = pd.read_csv(r'C:\Users\HAMED\miniconda3\MyCSV-FX\BTC-3years.csv')
df.head()

In [ ]:
df['Close'] = df[['Close']].apply(lambda x: x.str.replace(',', '.').astype(float), axis=1)
df = df.set_index("Date")[['Close']]
df = df.set_index(pd.to_datetime(df.index))
df = df[::-1]
df.head()

In [ ]:
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns , index = df.index)
df.head()

In [ ]:
df.plot(figsize=(18,8))
plt.title('BTC Prices')
plt.ylabel('Normalised Prices')
plt.show()

In [ ]:
def split_sequences(seq, n_steps_in , n_steps_out):  # X = inputs / Y = results
    X,y = [],[]
    for i in range(len(seq)):
        end = i + n_steps_in
        out_end = end + n_steps_out
        
        
        if out_end > len(seq):
            break
            
        seq_x ,seq_y = seq[i:end] , seq[end:out_end]
    
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X) , np.array(y)

In [ ]:
def visualize_results(results):
    history = results.history
    plt.figure(figsize=(12,4))
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss','loss'])
    plt.title('loss')
    plt.xlabel('Epoches')
    plt.ylabel('loss')
    plt.show()
    
    history = results.history
    plt.figure(figsize=(12,4))
    plt.plot(history['val_acc'])
    plt.plot(history['acc'])
    plt.legend(['val_acc','acc'])
    plt.title('accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.show()

In [ ]:
n_per_in = 30
n_per_out = 10
n_features = 1

X,y = split_sequences(list(df.Close), n_per_in ,n_per_out )

print(X.shape)

X = X.reshape(X.shape[0],X.shape[1],n_features)

print(X.shape)

In [ ]:
model = Sequential()
model.add(LSTM(96 , return_sequences=True, input_shape=(n_per_in,n_features)))
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96, return_sequences=True)) #hidden layer
model.add(LSTM(96 ))   #output layer
model.add(Dense(n_per_out, activation ='sigmoid')) #should have the same amount as we want to predict

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['acc'])  #MSE is a numerical feature

In [ ]:
if (not os.path.exists(r'C:\Users\HAMED\miniconda3\myLSTMNN\prediction_BTC_2h.h5')) :
    res = model.fit(X,y, epochs=2000 ,batch_size=327, validation_split=0.33)
    model.save(r'C:\Users\HAMED\miniconda3\myLSTMNN\prediction_BTC_2h.h5')

In [ ]:
model_res = load_model(r'C:\Users\HAMED\miniconda3\myLSTMNN\prediction_BTC_2h.h5')

In [ ]:
loss, acc = model_res.evaluate(X, y, verbose=0)
print( 'loss: %f, acc: %f' % (loss, acc*100))

In [ ]:
visualize_results(res)

In [ ]:
plt.figure(figsize=(12,4))

yhat = model.predict(X[-1].reshape(1,n_per_in,n_features)).tolist()[0]

yhat =scaler.inverse_transform(np.array(yhat).reshape(-1,1)).tolist()

actual =scaler.inverse_transform(y[-1].reshape(-1,1))

print('predicted' , yhat)
plt.plot(yhat , label='predicted')

print('actual', actual.tolist())

plt.plot(actual.tolist(), label='actual')

plt.title('Predicted vs Actual')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
yhat = model.predict(np.array(df.tail(n_per_in)).reshape(1,n_per_in,n_features)).tolist()
yhat = scaler.inverse_transform(np.array(yhat).reshape(-1,1)).tolist()

preds = pd.DataFrame(yhat , index= pd.date_range(start=df.index[-1], periods=len(yhat), freq="D"), columns=df.columns)

print(preds)

periods = 10

actual = pd.DataFrame(scaler.inverse_transform(df[['Close']].tail(periods)) , index = df.Close.tail(periods).index, columns=df.columns).append(preds.head(1))

plt.figure(figsize=(12,4))
plt.plot(actual , label = "actual price")
plt.plot(preds , label="predicted price")


plt.ylabel('price')
plt.xlabel('Date')

plt.legend()
plt.show()